In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Particionado") \
    .master("local[5]").getOrCreate()

24/01/18 16:53:36 WARN Utils: Your hostname, carloscastau resolves to a loopback address: 127.0.1.1; using 192.168.0.13 instead (on interface enp3s0)
24/01/18 16:53:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/18 16:53:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.range(0,20)
df.rdd.getNumPartitions()

5

In [3]:
rdd1 = spark.sparkContext.parallelize((0,20), 10)
rdd1.getNumPartitions()

10

In [4]:
rddDesdeArchivo = spark \
    .sparkContext \
    .textFile("./files/deporte.csv", 10)

In [5]:
rddDesdeArchivo.getNumPartitions()

10

In [6]:
rddDesdeArchivo.saveAsTextFile("./partition_example/salidatexto")

In [7]:
!ls "./partition_example/salidatexto/"

part-00000  part-00002	part-00004  part-00006	part-00008  _SUCCESS
part-00001  part-00003	part-00005  part-00007	part-00009


SUCCESS es un archivo tipo log que si no hubo error siempre debe venir vacio

In [8]:
# Verificamos los archivos particionados
!head -n 5 ./partition_example/salidatexto/part-00004

29,Cycling
30,Diving
31,Canoeing
32,Tennis
33,Modern Pentathlon


Cargando los archivos particionados en un RDD

In [9]:
rdd = spark.sparkContext.wholeTextFiles('./partition_example/salidatexto/*')

In [10]:
rdd.take(2)

[('file:/home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/partition_example/salidatexto/part-00000',
  'deporte_id,deporte\n1,Basketball\n2,Judo\n3,Football\n4,Tug-Of-War\n5,Speed Skating\n6,Cross Country Skiing\n'),
 ('file:/home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/partition_example/salidatexto/part-00001',
  '7,Athletics\n8,Ice Hockey\n9,Swimming\n10,Badminton\n11,Sailing\n12,Biathlon\n13,Gymnastics\n14,Art Competitions\n')]

In [11]:
# forma sencilla cargar
lista = rdd.mapValues(lambda x: x.split()).collect()

In [12]:
lista

[('file:/home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/partition_example/salidatexto/part-00000',
  ['deporte_id,deporte',
   '1,Basketball',
   '2,Judo',
   '3,Football',
   '4,Tug-Of-War',
   '5,Speed',
   'Skating',
   '6,Cross',
   'Country',
   'Skiing']),
 ('file:/home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/partition_example/salidatexto/part-00001',
  ['7,Athletics',
   '8,Ice',
   'Hockey',
   '9,Swimming',
   '10,Badminton',
   '11,Sailing',
   '12,Biathlon',
   '13,Gymnastics',
   '14,Art',
   'Competitions']),
 ('file:/home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/partition_example/salidatexto/part-00002',
  ['15,Alpine',
   'Skiing',
   '16,Handball',
   '17,Weightlifting',
   '18,Wrestling',
   '19,Luge',
   '20,Water',
   'Polo']),
 ('file:/home/carloscastau/Documents/platzi/data_engineer/curso-apache-spark-platzi-master/partition_example/salidatexto/part-00003',
 

In [13]:
lista = [l[0] for l in lista]
lista.sort()

In [14]:
rdd2 = spark \
    .sparkContext \
    .textFile(','.join(lista), 10 ).map(lambda l: l.split(","))

In [15]:
rdd2.take(5)

[['deporte_id', 'deporte'],
 ['1', 'Basketball'],
 ['2', 'Judo'],
 ['3', 'Football'],
 ['4', 'Tug-Of-War']]